In [ ]:
#Transfer-Learning on CIFAR 100 dataset using resnet50

In [3]:
#import statements
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [1]:

#preprocessing 

def preprocessor(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def prep(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return prep

In [4]:
num_classes = 100
nb_epochs = 10

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

#Pre-process the data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

datagen = ImageDataGenerator(preprocessing_function=preprocessor(v_l=0, v_h=1, pixel_level=True))
datagen.fit(x_train)

169009152/169001437 [==============================] - 2s 0us/step


In [ ]:
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
#Load a pretrained ResNet50 model without last FC layer (include_top=False) for Imagenet.
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
#Trained for 10 epochs with BatchSize of 64 
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

t=time.time()
historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                  batch_size=64),
                                  steps_per_epoch=x_train.shape[0] // 64,
                                  epochs=25,
                                  validation_data=(x_test, y_test))
print('Training time: %s' % (t - time.time()))

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
781/781 [==============================] - 261s 334ms/step - loss: 2.0154 - accuracy: 0.4812 - val_loss: 1.0100 - val_accuracy: 0.7011
Epoch 2/25
781/781 [==============================] - 259s 332ms/step - loss: 1.1678 - accuracy: 0.6640 - val_loss: 0.8551 - val_accuracy: 0.7427
Epoch 3/25
781/781 [==============================] - 259s 332ms/step - loss: 0.9783 - accuracy: 0.7126 - val_loss: 0.7779 - val_accuracy: 0.7649
Epoch 4/25
781/781 [==============================] - 259s 332ms/step - loss: 0.8746 - accuracy: 0.7380 - val_loss: 0.7438 - val_accuracy: 0.7723
Epoch 5/25
781/781 [==============================] - 260s 332ms/step - loss: 0.8074 - accuracy: 0.7588 - val_loss: 0.7248 - val_accuracy: 0.7830
Epoch 6/25
781/781 [==============================] - 260s 333ms/step - loss: 0.7461 - accuracy: 0.7731 - val_loss: 0.7366 - val_accuracy: 0.7804
Epoch 7/25
781/781 [============================

In [ ]:

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d (UpSampling2D) (None, None, None, None)  0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, None, None, None)  0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0